In [ ]:
import torch
import torchvision
import torchvision.transforms as tvt
from torch.utils.data import DataLoader
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')
from Fairnet import FairNetViT,FairNetCelebATrainer,evaluate_model,print_metrics,BaselineViT

def seed_everything(seed):
    """设置随机种子"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 设备配置
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
seed_everything(45)

# 数据集参数
image_size = 64
batch_size = 128

# 数据加载
print("Loading CelebA dataset...")
dataset = torchvision.datasets.CelebA(
    "./celebA/", 
    split='train', 
    transform=tvt.Compose([
        tvt.Resize((image_size, image_size)),
        tvt.ToTensor(),
        tvt.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
)

test_dataset = torchvision.datasets.CelebA(
    "./celebA/", 
    split='test', 
    transform=tvt.Compose([
        tvt.Resize((image_size, image_size)),
        tvt.ToTensor(),
        tvt.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
)

training_data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
print(f"Training samples: {len(dataset)}, Test samples: {len(test_dataset)}")

def demo_celeba():
    """
    CelebA 数据集上的 FairNet 演示
    
    对比基线模型和 FairNet 的性能，特别关注少数群体（Blond Male）
    """
    print("="*60)
    print("FairNet Demo on CelebA Dataset")
    print("="*60)
    print(f"\nDevice: {device}")
    print(f"Task: Gender Classification (Male/Female)")
    print(f"Sensitive Attribute: Hair Color (Blond/Non-Blond)")
    print(f"Minority Group: Blond Male")
    
    # ViT 配置
    vit_config = ViTConfig(
        num_hidden_layers=8,
        num_attention_heads=8,
        intermediate_size=768,
        image_size=64,
        patch_size=16
    )
    
    # ================================================================
    # 实验 1：基线模型
    # ================================================================
    print("\n" + "#"*60)
    print("# Experiment 1: Baseline ViT")
    print("#"*60)
    
    baseline_model = BaselineViT(vit_config)
    baseline_model = train_baseline(
        baseline_model, 
        training_data_loader, 
        device,
        epochs=15,
        lr=1e-5
    )
    
    baseline_metrics = evaluate_model(
        baseline_model, 
        test_data_loader, 
        device,
        model_type='baseline'
    )
    print_metrics(baseline_metrics, "Baseline ViT Results")
    
    # ================================================================
    # 实验 2：FairNet
    # ================================================================
    print("\n" + "#"*60)
    print("# Experiment 2: FairNet ViT")
    print("#"*60)
    
    fairnet_model = FairNetViT(
        vit_config,
        lora_rank=8,
        lora_alpha=16.0,
        lora_threshold=0.5,
        detector_hidden=128
    )
    
    trainer = FairNetCelebATrainer(fairnet_model, device)
    
    # 四阶段训练
    trainer.train_full(
        training_data_loader,
        stage1_epochs=10,      # 基础模型训练
        stage2_epochs=5,       # 检测器训练
        stage4_epochs=10,      # LoRA 训练
        stage1_lr=1e-5,
        stage2_lr=1e-4,
        stage4_lr=1e-4
    )
    
    fairnet_metrics = evaluate_model(
        fairnet_model, 
        test_data_loader, 
        device,
        model_type='fairnet'
    )
    print_metrics(fairnet_metrics, "FairNet ViT Results")
    
    # ================================================================
    # 结果对比
    # ================================================================
    print("\n" + "#"*60)
    print("# Comparison: Baseline vs FairNet")
    print("#"*60)
    
    print("\n📈 Performance Comparison:")
    print("-" * 50)
    print(f"{'Metric':<30} {'Baseline':<12} {'FairNet':<12} {'Δ':<10}")
    print("-" * 50)
    
    comparison_metrics = [
        ('accuracy', 'Overall Accuracy'),
        ('balanced_accuracy', 'Balanced Accuracy'),
        ('acc_non_blond_female', 'Non-Blond Female Acc'),
        ('acc_non_blond_male', 'Non-Blond Male Acc'),
        ('acc_blond_female', 'Blond Female Acc'),
        ('acc_blond_male', 'Blond Male Acc ⭐'),
        ('worst_group_accuracy', 'Worst Group Acc'),
        ('EOP', 'Equal Opportunity'),
        ('EOD', 'Equalized Odds'),
    ]
    
    for key, name in comparison_metrics:
        base_val = baseline_metrics.get(key, 0)
        fair_val = fairnet_metrics.get(key, 0)
        delta = fair_val - base_val
        
        # 对于公平性指标，减小是好事
        if key in ['EOP', 'EOD', 'accuracy_gap']:
            delta_str = f"{delta:+.4f}" if delta <= 0 else f"{delta:+.4f}"
            improvement = "✓" if delta < 0 else ""
        else:
            delta_str = f"{delta:+.4f}"
            improvement = "✓" if delta > 0 else ""
        
        print(f"{name:<30} {base_val:<12.4f} {fair_val:<12.4f} {delta_str} {improvement}")
    
    print("-" * 50)
    
    # 关键改进总结
    blond_male_improvement = (
        fairnet_metrics.get('acc_blond_male', 0) - 
        baseline_metrics.get('acc_blond_male', 0)
    )
    majority_change = (
        fairnet_metrics.get('acc_non_blond_female', 0) - 
        baseline_metrics.get('acc_non_blond_female', 0)
    )
    
    print("\n🎯 Key Findings:")
    print(f"  • Blond Male (Minority) Improvement: {blond_male_improvement:+.4f}")
    print(f"  • Non-Blond Female (Majority) Change: {majority_change:+.4f}")
    
    if blond_male_improvement > 0 and majority_change >= -0.01:
        print("\n✅ SUCCESS: FairNet improved minority group performance "
              "without significant loss on majority group!")
    elif blond_male_improvement > 0:
        print("\n⚠️ PARTIAL SUCCESS: Minority improved but some majority impact")
    else:
        print("\n❌ Need tuning: Consider adjusting hyperparameters")
    
    return baseline_model, fairnet_model, baseline_metrics, fairnet_metrics


# ============================================================================
# 运行演示
# ============================================================================

if __name__ == "__main__":
    baseline_model, fairnet_model, baseline_metrics, fairnet_metrics = demo_celeba()
    
    # 保存模型
    print("\nSaving models...")
    torch.save(baseline_model.state_dict(), "baseline_vit_celeba.pth")
    torch.save(fairnet_model.state_dict(), "fairnet_vit_celeba.pth")
    print("Models saved!")

Loading CelebA dataset...
Training samples: 162770, Test samples: 19962
FairNet Demo on CelebA Dataset

Device: cuda:1
Task: Gender Classification (Male/Female)
Sensitive Attribute: Hair Color (Blond/Non-Blond)
Minority Group: Blond Male

############################################################
# Experiment 1: Baseline ViT
############################################################

Training Baseline ViT Model


Epoch 1/15: 100%|██████████| 1271/1271 [02:42<00:00,  7.83it/s, loss=0.1757]


Epoch 1: Avg Loss = 0.2661


Epoch 2/15: 100%|██████████| 1271/1271 [02:40<00:00,  7.91it/s, loss=0.1051]


Epoch 2: Avg Loss = 0.1481


...

Epoch 9: Loss=0.0017, C_Loss=0.0010, T_Loss=0.0073


Epoch 10/10: 100%|██████████| 1271/1271 [02:12<00:00,  9.61it/s, loss=0.0004, c_loss=0.0000, t_loss=0.0044]


Epoch 10: Loss=0.0019, C_Loss=0.0013, T_Loss=0.0064


Evaluating: 100%|██████████| 156/156 [00:15<00:00,  9.92it/s]



FairNet ViT Results

📊 Overall Performance:
  Accuracy: 0.9559
  Balanced Accuracy: 0.9576

👥 Group Performance:
  Non-Blond Female: 0.9431 (n=9767)
  Non-Blond Male:   0.9676 (n=7535)
  Blond Female:     0.9782 (n=2480)
  Blond Male:       0.8556 (n=180) ⬅️ Minority Group

🎯 Worst Group Accuracy: 0.8556

⚖️ Fairness Metrics:
  EOP (Equal Opportunity): 0.1121
  EOD (Equalized Odds):    0.0736
  Accuracy Gap:            0.1121

🔧 LoRA Activation Rate: 0.1265

############################################################
# Comparison: Baseline vs FairNet
############################################################

📈 Performance Comparison:
--------------------------------------------------
Metric                         Baseline     FairNet      Δ         
--------------------------------------------------
Overall Accuracy               0.9582       0.9559       -0.0023 
Balanced Accuracy              0.9569       0.9576       +0.0007 ✓
Non-Blond Female Acc           0.9568       0.9431